In [3]:
algos = ['LightGBM', 'Xgboost', 'CatBoost']

from supervised.automl import AutoML

from helpers import *

training_filters = [
    {
        'feature': 'month',
        'operator': 'between',
        'value': [3, 9]
    }
]

validation_filters = [
    {
        'feature': 'date_forecast',
        'operator': 'between',
        'value': ['2020-05-01', '2020-07-01']
    }
]

test_filters = [
    {
        'feature': 'date_forecast',
        'operator': 'between',
        'value': ['2021-05-01', '2021-07-01']
    }
]


agg_strats = ['min','max','mean']

m_wrap = DataWrapper(
    impute_strategy = 'fbfill',
    aggregation_strats = agg_strats,
    y_scalers = [Y_Scaler_MaxAbs_per_building()],
    transform_pred_strategy = [],
    training_filters=training_filters,
    validation_filters=validation_filters,
    test_filters=test_filters,
    bagging_filters=False, # these are applied additionally to the other filters on val, test, sub
    )

X_train_agg, y_train_agg = m_wrap.get_train(aggregated=True,drop_val=False,drop_test=False,y_scaled=True)
X_sub_agg = m_wrap.get_sub(aggregated=True)

train_idx = X_train_agg[~X_train_agg['date_forecast'].between('2020-05-01', '2020-07-01')].index
val_idx = X_train_agg[X_train_agg['date_forecast'].between('2020-05-01', '2020-07-01')].index

X_train_agg = X_train_agg.select_dtypes(exclude=['datetime','timedelta','period[M]'])

cv = [(train_idx, val_idx)]

mod = AutoML(
    results_path='AutoML_Results_XGBOOST/',

)

pred = mod.predict(X_sub_agg)
pred

{'a': 5733.42, 'b': 1152.3, 'c': 999.6}
{'a': 5733.42, 'b': 1152.3, 'c': 999.6}


/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MiniBatchKMeans from version 1.3.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
2023-11-11 23:50:28,382 supervised.exceptions ERROR Missing column: min_effective_cloud_cover:p_-1h in input data. Cannot predict


AutoMLException: Missing column: min_effective_cloud_cover:p_-1h in input data. Cannot predict